In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 10

## 11.1. 날짜, 시간자료형, 도구  
datetime,time,calande모듈

In [2]:
from datetime import datetime
now = datetime.now()
now
now.year;now.month;now.day

datetime.datetime(2019, 11, 30, 16, 13, 18, 454269)

2019

11

30

datetime은 날짜와 시간을 모두 저장하며, 마이크로초까지 지원함

In [3]:
now.second;now.microsecond
"mili:{}".format(now.microsecond//1000)
"micro:{}".format(now.microsecond%1000)


18

454269

'mili:454'

'micro:269'

datetime.timedelta는 두 객체간 시간적 차이를 표현가능

In [4]:
delta = datetime(2011,1,7) - datetime(2008,6,24,8,15)
delta
delta.days
delta.seconds

datetime.timedelta(days=926, seconds=56700)

926

56700

timedelta를 더하거나 빼면 그만큼의 시간이 datetime객체에 추가됨

In [5]:
from datetime import timedelta
start = datetime(2011,1,7)
start + timedelta(12)
start - 2*timedelta(2)

datetime.datetime(2011, 1, 19, 0, 0)

datetime.datetime(2011, 1, 3, 0, 0)

### 11.1.1 문자열을 datetime으로 변환하기

datetime객체와 pandas의 Timestamp 객체는 str메서드나 strftime 메서드에 규칙을 넘겨 문자열로 표현가능

In [6]:
stamp = datetime(2011,1,3)
stamp;str(stamp)
stamp.strftime('%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

'2011-01-03 00:00:00'

'2011-01-03'

strptime 메서드를 이용하면 문자열을 다시 시간으로 변경도 가능함

In [7]:
value = '08:15:01'
#datetime.strptime(value,'%H:%M:%S')
datetime.strptime(value,'%H:%M:%S')
datetime.strptime(value,'%H:%M:%S').time()

value = '2011-01-03'
datetime.strptime(value,'%Y-%m-%d')

datetime.datetime(1900, 1, 1, 8, 15, 1)

datetime.time(8, 15, 1)

datetime.datetime(2011, 1, 3, 0, 0)

In [8]:
datestrs = ['7/6/2011','8/6/2011']
[datetime.strptime(d,'%d/%m/%Y') for d in datestrs]

[datetime.datetime(2011, 6, 7, 0, 0), datetime.datetime(2011, 6, 8, 0, 0)]

strptime 메서드는 알려진형식의 날짜를 파싱하는 최적의 방법이기는 하지만 매번 포맷규칙을 쓰기 귀찮음  
dateutil에 포함된 parser.parse 메서드를 사용하면됨 (판다스깔때 자동으로 설치됨)

사람이 인지하는 대부분의 날짜형식을 파싱할수 있음

In [9]:
from dateutil.parser import parse
parse('2011-05-14')
parse('5/15/19')
parse('15/5/19')
parse('May,31,19')
parse('july 16,16')

datetime.datetime(2011, 5, 14, 0, 0)

datetime.datetime(2019, 5, 15, 0, 0)

datetime.datetime(2019, 5, 15, 0, 0)

datetime.datetime(2019, 5, 31, 0, 0)

datetime.datetime(2019, 7, 16, 0, 0)

국제로케이션에서 날짜가 월 앞에 오는 경우 dayfirst=True 인자를 넘겨주면됨

In [10]:
parse('5/18/19',dayfirst=True)

datetime.datetime(2019, 5, 18, 0, 0)

pandas는 일반적으로 dataframe의 컬럼이나 축 색인으로 날짜가 담길 배열을 사용함  
to_datetime메서드를 이용하면 많은 종류의 표현을 처리할수 있고, 표준형식은 매우 빠르게 처리함

In [11]:
datestrs = ['2011/7/6 12:00:00','2011-08-06 00:00:00']
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

누락값 none으로 간주되어야할 값도 처리해줌

In [12]:
idx = pd.to_datetime(datestrs + [None])
idx
pd.isnull(idx) #Not a time

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

array([False, False,  True])

## 11.2. 시계열 기초  
가장 기본적인 시계열 객체는 문자열이나 datetime객체로 표현되는 타임스탬프 Series

In [13]:
from datetime import datetime
dates = [datetime(2011,1,2),datetime(2011,1,5),
        datetime(2011,1,7),datetime(2011,1,8),
        datetime(2011,1,10),datetime(2011,1,12)]
ts = pd.Series(np.random.randn(6),index = dates)
ts
ts.index
type(ts.index)

2011-01-02   -0.133197
2011-01-05    0.970021
2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

pandas.core.indexes.datetimes.DatetimeIndex

numpy의 datetime64를 사용해 나노초 단위로 타임스탬프 저장  
DatetimeIndex의 스칼라값은 Timestamp 객체

In [14]:
ts.index.dtype
ts.index[0]

dtype('<M8[ns]')

Timestamp('2011-01-02 00:00:00')

In [15]:
ts + ts[::2]

2011-01-02   -0.266395
2011-01-05         NaN
2011-01-07   -2.067086
2011-01-08         NaN
2011-01-10    0.013637
2011-01-12         NaN
dtype: float64

### 11.2.1 색인, 선택, 부분선택

시계열의 인덱싱은 시리즈와 동일함

In [16]:
ts
ts.index[2]
ts[ts.index[2]]

2011-01-02   -0.133197
2011-01-05    0.970021
2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

Timestamp('2011-01-07 00:00:00')

-1.0335432367560422

해석가능한 문자를 넘겨서 편리하게 사용할 수 있음

In [17]:
ts['1/7/2011']
ts['20110107']

-1.0335432367560422

-1.0335432367560422

긴 시계열에서 연이나 월만 넘겨서 데이터 일부를 선택할수도있음

In [18]:
longer_ts = pd.Series(np.random.randn(1000),
                     index = pd.date_range('1/1/2000',periods=1000))
longer_ts
longer_ts['2001']

2000-01-01   -0.434839
2000-01-02   -1.193834
2000-01-03   -0.910135
2000-01-04   -0.953573
2000-01-05   -0.317469
                ...   
2002-09-22    0.340627
2002-09-23    0.707426
2002-09-24   -0.487762
2002-09-25    1.279351
2002-09-26   -0.183709
Freq: D, Length: 1000, dtype: float64

2001-01-01   -1.484754
2001-01-02    2.638354
2001-01-03   -0.402374
2001-01-04    0.725776
2001-01-05    0.030454
                ...   
2001-12-27    0.504657
2001-12-28   -0.405505
2001-12-29   -0.249038
2001-12-30    0.286142
2001-12-31    0.113008
Freq: D, Length: 365, dtype: float64

In [19]:
longer_ts['2001-05']

2001-05-01   -0.798037
2001-05-02    0.897970
2001-05-03   -0.311906
2001-05-04   -0.324891
2001-05-05   -2.367843
                ...   
2001-05-27   -0.816132
2001-05-28    0.950845
2001-05-29    2.412698
2001-05-30    1.365216
2001-05-31   -0.492900
Freq: D, Length: 31, dtype: float64

슬라이싱도 유사함

In [20]:
ts[datetime(2011,1,7):]

2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

In [21]:
ts
ts['1/6/2011':'01/11/2011']

2011-01-02   -0.133197
2011-01-05    0.970021
2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
dtype: float64

슬라이스된 데이터는 원본의 뷰이므로 변경사항이 원본에 적용된다는것에 유의!

truncate 인스턴스 메서드는 TimeSeries를 두개의 날짜로 나눔

In [22]:
ts
ts.truncate(before='1/9/2011')
ts.truncate(after='1/9/2011')

2011-01-02   -0.133197
2011-01-05    0.970021
2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

2011-01-02   -0.133197
2011-01-05    0.970021
2011-01-07   -1.033543
2011-01-08   -1.501971
dtype: float64

**DataFrame에도 동일하게 적용되며, 로우에 인덱싱됨**

In [23]:
dates = pd.date_range('1/1/2000',periods = 100,freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100,4),
                      index = dates,
                      columns = ['Colorado','Texas','New York','Ohio'])
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,0.099916,-1.000112,-0.550672,1.106278
2001-05-09,0.531659,-1.143307,-1.266565,-0.444126
2001-05-16,2.219873,0.594260,1.310923,1.768362
2001-05-23,0.243243,0.521587,-0.335705,0.930394
2001-05-30,-0.139652,1.465411,-0.343443,-1.138843


### 11.2.2. 중복된 색인을 갖는 시계열

In [24]:
dates = pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000','1/3/2000'])
dup_ts = pd.Series(np.arange(5),index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

is_unique 속성을 보면 색인이 유일하지 않음을 확인가능

In [25]:
dup_ts.index.duplicated()

array([False, False,  True,  True, False])

In [26]:
dup_ts.index.is_unique

False

이걸 인덱싱하면 중복여부에 따라 스칼라 혹은 슬라이스가 생성

In [27]:
dup_ts['1/3/2000']
dup_ts['1/2/2000']

4

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

유일하지 않은 타임스탬프를 가진 데이터를 집계한다고 하면, 한가지 방법은 groupby에 level=0(단일단계 인덱싱)을 넘기는것

In [28]:
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [29]:
# dup_ts.groupby(dup_ts.index).mean()
dup_ts.groupby(level=0).mean()
dup_ts.groupby(level=0).count()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3. 날짜 범위, 빈도, 이동  
빈도간 변환이나 리샘플링은 큰주제이므로 11.6절에서 따로 진행함

In [30]:
#시계열을 고정된 일 빈도로 변환하기 (resample)
ts
resampler = ts.resample('D')
resampler

2011-01-02   -0.133197
2011-01-05    0.970021
2011-01-07   -1.033543
2011-01-08   -1.501971
2011-01-10    0.006818
2011-01-12   -0.325214
dtype: float64

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

### 11.3.1. 날짜 범위 생성하기  
pandas.date_range를 사용하면 특정 빈도에 따라 지정한 길이만큼의 DatetimeIndex를 생성가능

In [31]:
index = pd.date_range('2012-04-01','2012-06-01')
index
'''기본적으로 일별 타임스탬프 생성'''

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

'기본적으로 일별 타임스탬프 생성'

In [32]:
#시작날짜나 종료날짜만 넘기다면 생성할 기간의 숫자를 함께 전달해야함
pd.date_range(start = '2012-04-01',periods=20)
pd.date_range(end = '2012-04-01',periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2012-03-13', '2012-03-14', '2012-03-15', '2012-03-16',
               '2012-03-17', '2012-03-18', '2012-03-19', '2012-03-20',
               '2012-03-21', '2012-03-22', '2012-03-23', '2012-03-24',
               '2012-03-25', '2012-03-26', '2012-03-27', '2012-03-28',
               '2012-03-29', '2012-03-30', '2012-03-31', '2012-04-01'],
              dtype='datetime64[ns]', freq='D')

In [33]:
pd.date_range('2000-01-01','2000-12-01',freq='BM') #해당 기간내,영업마감일만 표시하기

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [34]:
#시작이나 종료시간의 타임스탬프를 보존함
pd.date_range('2000-01-01 12:00:24',periods=10)
pd.date_range(end='2000-01-01 12:10:24',periods=10)
pd.date_range('2000-01-01 12:00:24','2000-01-05 11:20:24')

DatetimeIndex(['2000-01-01 12:00:24', '2000-01-02 12:00:24',
               '2000-01-03 12:00:24', '2000-01-04 12:00:24',
               '2000-01-05 12:00:24', '2000-01-06 12:00:24',
               '2000-01-07 12:00:24', '2000-01-08 12:00:24',
               '2000-01-09 12:00:24', '2000-01-10 12:00:24'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['1999-12-23 12:10:24', '1999-12-24 12:10:24',
               '1999-12-25 12:10:24', '1999-12-26 12:10:24',
               '1999-12-27 12:10:24', '1999-12-28 12:10:24',
               '1999-12-29 12:10:24', '1999-12-30 12:10:24',
               '1999-12-31 12:10:24', '2000-01-01 12:10:24'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2000-01-01 12:00:24', '2000-01-02 12:00:24',
               '2000-01-03 12:00:24', '2000-01-04 12:00:24'],
              dtype='datetime64[ns]', freq='D')

시간정보도 포함하여 갖고있지만, 관례에 맞추어 자정에 맞추어 타임스탬프를 정규화 하고 싶다면 normalize 옵션 사용

In [35]:
pd.date_range('2012-05-02 12:51:31',periods=5,normalize = True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 11.3.2. 빈도와 날짜 옵셋  
pandas에서 빈도는 기본빈도와 배수의 조합으로 이루어짐  
각각의 기본빈도는 날짜오프셋이라는 객체를 사용가능

In [36]:
from pandas.tseries.offsets import Hour,Minute
hour=  Hour()
hour

<Hour>

In [37]:
four_hours = Hour(4)
four_hours

<4 * Hours>

대부분의 어플리케이션에서는 위처럼 직접 객체를 만들 경우는 없고 'H','4H'처럼 문자열로 표현

In [38]:
pd.date_range('2001-01-01','2001-01-03',freq='4H')

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 04:00:00',
               '2001-01-01 08:00:00', '2001-01-01 12:00:00',
               '2001-01-01 16:00:00', '2001-01-01 20:00:00',
               '2001-01-02 00:00:00', '2001-01-02 04:00:00',
               '2001-01-02 08:00:00', '2001-01-02 12:00:00',
               '2001-01-02 16:00:00', '2001-01-02 20:00:00',
               '2001-01-03 00:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [39]:
#여러 옵셋을 덧셈으로 합치기
Hour(2)+Minute(30)

#문자열로 넘겨도 같은 표현으로 해석됨
pd.date_range('2001-01-01','2001-01-02',freq='1h30min')

<150 * Minutes>

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 01:30:00',
               '2001-01-01 03:00:00', '2001-01-01 04:30:00',
               '2001-01-01 06:00:00', '2001-01-01 07:30:00',
               '2001-01-01 09:00:00', '2001-01-01 10:30:00',
               '2001-01-01 12:00:00', '2001-01-01 13:30:00',
               '2001-01-01 15:00:00', '2001-01-01 16:30:00',
               '2001-01-01 18:00:00', '2001-01-01 19:30:00',
               '2001-01-01 21:00:00', '2001-01-01 22:30:00',
               '2001-01-02 00:00:00'],
              dtype='datetime64[ns]', freq='90T')

월 마지막일 이나 월 영업 마감일은 월말이 주말인지 아닌지에 따라 다르므로 따로 앵커드 옵셋이라 부르겠음

**월별 주차(WOM)**  
이걸 이용하면 매월 셋째주 금요일 등을 얻을수 있음

In [40]:
rng = pd.date_range('2012-01-01','2012-09-01',freq='WOM-3FRI')
rng

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

### 11.3.3. 데이터 시프트  
데이터를 시간축에서 앞이나 뒤로 이동하는것(느슨한 시프트)

In [41]:
ts = pd.Series(np.random.randn(4),index=pd.date_range('1/1/2000',periods=4,freq='M'))
ts
ts.shift(2)
ts.shift(-2)

2000-01-31    0.428187
2000-02-29   -0.029414
2000-03-31   -0.208472
2000-04-30    0.474000
Freq: M, dtype: float64

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.428187
2000-04-30   -0.029414
Freq: M, dtype: float64

2000-01-31   -0.208472
2000-02-29    0.474000
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

시프트는 한시계열 내에서 퍼센트 변화를 계산할때 흔히 사용함

In [42]:
ts/ts.shift(1)-1

2000-01-31         NaN
2000-02-29   -1.068693
2000-03-31    6.087604
2000-04-30   -3.273682
Freq: M, dtype: float64

느슨한 시프트는 색인을 안바꾸니까 어떤 데이터는 버려짐,  
빈도를 알고있다면 shift에 빈도를 넣어서 타임스태프를 확장할수있음

In [43]:
ts
ts.shift(2)
ts.shift(2,freq='M')

2000-01-31    0.428187
2000-02-29   -0.029414
2000-03-31   -0.208472
2000-04-30    0.474000
Freq: M, dtype: float64

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.428187
2000-04-30   -0.029414
Freq: M, dtype: float64

2000-03-31    0.428187
2000-04-30   -0.029414
2000-05-31   -0.208472
2000-06-30    0.474000
Freq: M, dtype: float64

다른 빈도를 사용해도 되므로 유연하게 데이터를 밀고 당길 수 있음

In [44]:
ts
ts.shift(3,freq='D')
ts.shift(1,freq='90T') #T는 분

2000-01-31    0.428187
2000-02-29   -0.029414
2000-03-31   -0.208472
2000-04-30    0.474000
Freq: M, dtype: float64

2000-02-03    0.428187
2000-03-03   -0.029414
2000-04-03   -0.208472
2000-05-03    0.474000
dtype: float64

2000-01-31 01:30:00    0.428187
2000-02-29 01:30:00   -0.029414
2000-03-31 01:30:00   -0.208472
2000-04-30 01:30:00    0.474000
Freq: M, dtype: float64

**오프셋만큼 날짜 시프트하기**